# Can I plot the single-cell tracks and glimpses

Using the ground-truth checked tracks from the control (3,5) example.

In [1]:
import os
import btrack
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from macrohet import dataio, tools
from tqdm.auto import tqdm
from scipy import stats
import pickle
import re
import cv2
from skimage import io
colors = sns.set_palette('PiYG')
track_scale_factor = 5.04

### Load metadata

In [2]:
base_dir = '/mnt/DATA/macrohet/'
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

### Load all tracks

In [3]:
tracks_dict = dict()
### iterate over all experimental conditions
for (row, column), info in tqdm(assay_layout.iterrows(), 
                                desc = 'Progress through positions',
                                total = len(assay_layout)):

    ### load tracks
    with btrack.io.HDF5FileHandler(os.path.join(base_dir, 
                                                f'labels/macrohet_seg_model/{row,column}.h5'), 
                                       'r', 
                                       obj_type='obj_type_1'
                                       ) as reader:
        tracks = reader.tracks
            
    ### append tracks to dictionary
    tracks_dict[(row, column)] = tracks

Progress through positions:   0%|          | 0/24 [00:00<?, ?it/s]

[INFO][2023/06/22 02:58:54 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5...
[INFO][2023/06/22 02:58:54 PM] Loading tracks/obj_type_1
[INFO][2023/06/22 02:58:54 PM] Loading LBEP/obj_type_1
[INFO][2023/06/22 02:58:54 PM] Loading objects/obj_type_1 (39878, 5) (39878 filtered: None)
[INFO][2023/06/22 02:58:55 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5
[INFO][2023/06/22 02:58:55 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 5).h5...
[INFO][2023/06/22 02:58:55 PM] Loading tracks/obj_type_1
[INFO][2023/06/22 02:58:55 PM] Loading LBEP/obj_type_1
[INFO][2023/06/22 02:58:55 PM] Loading objects/obj_type_1 (41424, 5) (41424 filtered: None)
[INFO][2023/06/22 02:58:56 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 5).h5
[INFO][2023/06/22 02:58:56 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 6).h5...
[INFO][2023/06/22 02:58:56 PM] Loading tracks/obj_type_1
[INFO]

[INFO][2023/06/22 02:59:11 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 5).h5
[INFO][2023/06/22 02:59:11 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 6).h5...
[INFO][2023/06/22 02:59:11 PM] Loading tracks/obj_type_1
[INFO][2023/06/22 02:59:11 PM] Loading LBEP/obj_type_1
[INFO][2023/06/22 02:59:11 PM] Loading objects/obj_type_1 (45688, 5) (45688 filtered: None)
[INFO][2023/06/22 02:59:12 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 6).h5
[INFO][2023/06/22 02:59:12 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 7).h5...
[INFO][2023/06/22 02:59:12 PM] Loading tracks/obj_type_1
[INFO][2023/06/22 02:59:12 PM] Loading LBEP/obj_type_1
[INFO][2023/06/22 02:59:12 PM] Loading objects/obj_type_1 (45214, 5) (45214 filtered: None)
[INFO][2023/06/22 02:59:12 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 7).h5
[INFO][2023/06/22 02:59:12 PM] Opening HDF file: /mnt/DATA/macrohe

### Pick set of tracks of interest

In [5]:
row, column = 3, 5
tracks = tracks_dict[(row, column)]

### Load ground truth labels for tracks

In [6]:
gt_track_dict_fn = '/mnt/DATA/macrohet/upstream_development/tracking/tracking_performance/3,5/ground_truth_tracks/(3, 5)_track_assessment.json'
# Load the JSON data from the file
with open(gt_track_dict_fn, 'r') as file:
    gt_track_dict = json.load(file)

In [7]:
gt_track_dict

{'425': True,
 '694': True,
 '403': False,
 '427': True,
 '453': False,
 '217': True,
 '1002': False,
 '1893': False,
 '1986': False,
 '2192': False,
 '401': True,
 '412': False,
 '2499': False,
 '1446': False,
 '1003': True,
 '790': False,
 '1552': False,
 '2862': False,
 '432': True,
 '456': True,
 '461': True,
 '416': True,
 '1892': False,
 '430': True,
 '1226': True,
 '1783': True,
 '429,2818': 'Link',
 '431': True,
 '407': True,
 '2675,404': 'Link',
 '398': True,
 '423': True,
 '424': True,
 '428': False,
 '418': False,
 '411': False,
 '411,1683': 'Link',
 '1058': False,
 '466': True,
 '465': True,
 '397': True,
 '426': True,
 '419': True,
 '1735': True,
 '420': True,
 '421': True,
 '802': True,
 '455': False,
 '406': False,
 '410': True,
 '411,1118': 'Link',
 '1118': True,
 '1114,2130': 'Link',
 '414': True,
 '459': True,
 '2234': True,
 '470': True,
 '437': True,
 '1649,2453': 'Link',
 '2453,3126': 'Link',
 '444': True,
 '865': False,
 '441': True,
 '318': False,
 '2952': False,

In [8]:
true_IDs = [int(ID) for ID, status in gt_track_dict.items() if status is True]

### Filter true tracks

In [9]:
true_tracks = [t for t in tracks if t.ID in true_IDs]

In [10]:
true_tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy,major_axis_length,mean_intensity,minor_axis_length,Infected,orientation,area
0,423,0,723.909912,33.010132,0.000000,423,423,5,0,False,344.046997,"(3,) array",87.624420,1.0,-0.105442,23203.0
1,423,1,727.771362,38.211437,0.000000,423,423,5,0,False,410.356964,"(3,) array",95.545296,1.0,-0.105384,25659.0
2,423,2,730.887634,26.283424,0.000000,423,423,5,0,False,281.636444,"(3,) array",89.392372,1.0,-0.172310,19483.0
3,423,3,732.147583,27.895367,0.000000,423,423,5,0,False,325.991333,"(3,) array",92.556488,1.0,-0.172292,22840.0
4,423,4,730.891357,33.019173,0.000000,423,423,5,0,False,373.707001,"(3,) array",80.582886,1.0,-0.152742,23224.0
5,423,5,731.606384,30.630995,0.000000,423,423,5,0,False,351.986481,"(3,) array",78.869858,1.0,-0.151112,21159.0
6,423,6,732.788086,30.744682,0.000000,423,423,5,0,False,358.099487,"(3,) array",79.728958,1.0,-0.169930,21750.0
7,423,7,733.528625,27.149408,0.000000,423,423,5,0,False,294.958893,"(3,) array",67.987579,0.0,-0.195798,15457.0
8,423,8,733.524414,32.480957,0.000000,423,423,5,0,False,385.717529,"(3,) array",73.575928,1.0,-0.201907,21431.0
9,423,9,735.238953,29.900726,0.000000,423,423,5,0,False,409.529633,"(3,) array",79.070381,1.0,-0.251289,22385.0


# Load sc_df

In [15]:
# Specify the path to your pickle file
sc_df_path = "/mnt/DATA/macrohet/results/preliminary_sc_measures/sc_dfs/sc_df_>40.pkl"

# Load the pickle file
with open(sc_df_path, "rb") as file:
    df = pickle.load(file)

### Format df so that it can be read by Bokeh

In [19]:
# Remove non-letter characters and replace spaces with underscores in column names
df.columns = [re.sub(r'[^a-zA-Z ]', '', col).replace(' ', '_') for col in df.columns]

In [20]:
df

,Time_hours,x,y,x_scaled,y_scaled,Infection_status,Initial_infection_status,Final_infection_status,Area,Intracellular_mean_Mtb_content,...,delta_Mtb_max_foldchange_normalised_max_area,delta_Mtbdt,Eccentricity,MSD,Strain,Compound,Concentration,Cell_ID,Acquisition_ID,Unique_ID
0,0,708.125061,77.589088,3568.950308,391.049006,0.0,0.0,1.0,42556.0,365.408905,...,0.095232,0.418707,0.813310,0.000000,RD1,CTRL,EC0,433,"(3, 4)",433.3.4
1,1,725.449097,73.245003,3656.263447,369.154814,0.0,0.0,1.0,48727.0,362.852478,...,0.095232,0.418707,0.896474,17.860383,RD1,CTRL,EC0,433,"(3, 4)",433.3.4
2,2,723.088928,76.431496,3644.368198,385.214738,0.0,0.0,1.0,43484.0,363.515442,...,0.095232,0.418707,0.876911,3.965367,RD1,CTRL,EC0,433,"(3, 4)",433.3.4
3,3,726.009766,79.395073,3659.089219,400.151168,0.0,0.0,1.0,39113.0,360.448578,...,0.095232,0.418707,0.717504,4.161019,RD1,CTRL,EC0,433,"(3, 4)",433.3.4
4,4,728.209656,82.197769,3670.176665,414.276757,0.0,0.0,1.0,32644.0,362.289764,...,0.095232,0.418707,0.425791,3.562951,RD1,CTRL,EC0,433,"(3, 4)",433.3.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701474,70,757.886719,44.412071,3819.749063,223.836839,1.0,1.0,1.0,71808.0,358.460327,...,-0.000019,-0.092227,0.923363,0.522451,WT,BDQ,EC99,2320,"(6, 9)",2320.6.9
701475,71,760.801514,49.459522,3834.439629,249.275992,1.0,1.0,1.0,65328.0,358.026184,...,-0.000019,-0.092227,0.893735,5.828618,WT,BDQ,EC99,2320,"(6, 9)",2320.6.9
701476,72,761.494873,50.097046,3837.934160,252.489111,1.0,1.0,1.0,66829.0,358.846039,...,-0.000019,-0.092227,0.888738,0.941904,WT,BDQ,EC99,2320,"(6, 9)",2320.6.9
701477,73,759.911255,48.538654,3829.952725,244.634818,1.0,1.0,1.0,71877.0,358.451752,...,-0.000019,-0.092227,0.897926,2.221808,WT,BDQ,EC99,2320,"(6, 9)",2320.6.9


In [33]:
df.keys()

Index(['Time_hours', 'x', 'y', 'x_scaled', 'y_scaled', 'Infection_status',
       'Initial_infection_status', 'Final_infection_status', 'Area',
       'Intracellular_mean_Mtb_content',
       'Intracellular_thresholded_Mtb_content',
       'Intracellular_thresholded_Mtb_content_smooth',
       'Macroph_GFP_expression', 'delta_Mtb_raw', 'delta_Mtb_max_raw',
       'delta_Mtb_max_smooth', 'delta_Mtb_max_foldchange',
       'delta_Mtb_max_foldchange_normalised_mean_area',
       'delta_Mtb_max_foldchange_normalised_max_area', 'delta_Mtbdt',
       'Eccentricity', 'MSD', 'Strain', 'Compound', 'Concentration', 'Cell_ID',
       'Acquisition_ID', 'Unique_ID'],
      dtype='object')

### only use GT tracks for the moment

In [117]:
long_true_unique_IDs = [f'{ID}.3.5' for ID in true_IDs if f'{ID}.3.5' in df['Unique_ID'].unique()][0:20] #only do 20

In [118]:
subset_df = df[df['Unique_ID'].isin(long_true_unique_IDs)]

# Animating these graphs (making interactive)

In [32]:
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, output_file, ColumnDataSource, output_notebook
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
from bokeh.models import WheelZoomTool
from bokeh.plotting import save
from bokeh.models import Label
import matplotlib.cm as cm

In [62]:
subset_df['Color_codes'] = [(120, 197, 120) if infected == 'Uninfected' else (230, 97, 135) for infected in subset_df['Final_infection_status']]
subset_df['Color_codes'] = pd.Series(subset_df['Color_codes'])

/tmp/ipykernel_390854/1216087459.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['Color_codes'] = [(120, 197, 120) if infected == 'Uninfected' else (230, 97, 135) for infected in subset_df['Final_infection_status']]
/tmp/ipykernel_390854/1216087459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['Color_codes'] = pd.Series(subset_df['Color_codes'])


In [119]:
# Define the colormap and normalize the eccentricity values
cmap = cm.get_cmap('PiYG')
norm = plt.Normalize(vmin=subset_df['delta_Mtb_max_foldchange_normalised_max_area'].min(), vmax=subset_df['delta_Mtb_max_foldchange_normalised_max_area'].max())
# Create the "eccentricity_color_codes" column with RGB values
subset_df['color_codes'] = [
    tuple(int(255 * c) for c in cmap(norm(dmtb))[:3])
    for dmtb in subset_df['delta_Mtb_max_foldchange_normalised_max_area']
]

/tmp/ipykernel_390854/2192192939.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['color_codes'] = [


# Plot Mtb time series

In [123]:
ID_df

,Time_hours,x,y,x_scaled,y_scaled,Infection_status,Initial_infection_status,Final_infection_status,Area,Intracellular_mean_Mtb_content,...,delta_Mtbdt,Eccentricity,MSD,Strain,Compound,Concentration,Cell_ID,Acquisition_ID,Unique_ID,color_codes
26451,0,83.433136,62.470875,420.503005,314.853209,0.0,0.0,0.0,39074.0,354.011475,...,0.0,0.957862,0.000000,WT,CTRL,EC0,427,"(3, 5)",427.3.5,"(142, 1, 82)"
26452,1,84.654182,64.699432,426.657079,326.085139,0.0,0.0,0.0,32628.0,355.403168,...,0.0,0.966423,2.541146,WT,CTRL,EC0,427,"(3, 5)",427.3.5,"(142, 1, 82)"
26453,2,80.224953,76.636642,404.333762,386.248678,0.0,0.0,0.0,32707.0,355.472290,...,0.0,0.926621,12.732441,WT,CTRL,EC0,427,"(3, 5)",427.3.5,"(142, 1, 82)"
26454,3,78.774765,83.784775,397.024816,422.275265,0.0,0.0,0.0,25078.0,357.617188,...,0.0,0.911093,7.293753,WT,CTRL,EC0,427,"(3, 5)",427.3.5,"(142, 1, 82)"
26455,4,78.499710,88.938072,395.638539,448.247884,NaN,0.0,0.0,NaN,357.335490,...,0.0,0.892865,5.160633,WT,CTRL,EC0,427,"(3, 5)",427.3.5,"(142, 1, 82)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26521,70,129.611298,34.768669,653.240940,175.234092,0.0,0.0,0.0,21525.0,364.245392,...,0.0,0.801610,7.385997,WT,CTRL,EC0,427,"(3, 5)",427.3.5,"(142, 1, 82)"
26522,71,129.189804,30.401833,651.116613,153.225236,0.0,0.0,0.0,17200.0,364.277252,...,0.0,0.567481,4.387131,WT,CTRL,EC0,427,"(3, 5)",427.3.5,"(142, 1, 82)"
26523,72,131.427460,26.722143,662.394397,134.679602,0.0,0.0,0.0,18009.0,363.195740,...,0.0,0.815514,4.306648,WT,CTRL,EC0,427,"(3, 5)",427.3.5,"(142, 1, 82)"
26524,73,131.707245,31.655239,663.804514,159.542405,0.0,0.0,0.0,19550.0,362.451141,...,0.0,0.756540,4.941024,WT,CTRL,EC0,427,"(3, 5)",427.3.5,"(142, 1, 82)"


In [124]:
help(p.line)

Help on method line in module bokeh.plotting.glyph_api:

line(x='x', y='y', *, line_alpha=1.0, line_cap='butt', line_color='black', line_dash=[], line_dash_offset=0, line_join='bevel', line_width=1, name=None, syncable=True, tags=[], **kwargs) method of bokeh.plotting.figure.Figure instance
    Configure and add :class:`~bokeh.models.glyphs.Line` glyphs to this Figure.
    
    Args:
        x (:class:`~bokeh.core.properties.NumberSpec`\ , optional):
            The x-coordinates for the points of the line. (default: 'x')
    
        y (:class:`~bokeh.core.properties.NumberSpec`\ , optional):
            The y-coordinates for the points of the line. (default: 'y')
    
    
    Keyword args:
        line_alpha (:class:`~bokeh.core.properties.Alpha`\ , optional):
            The line alpha values for the line. (default: 1.0)
    
        line_cap (:class:`~bokeh.core.properties.Enum`\ (:class:`~bokeh.core.enums.LineCap`\ ), optional):
            The line cap values for the line. (defa

In [140]:
TOOLS = 'crosshair,save,pan,box_zoom,reset,wheel_zoom,hover'
p = figure(title="title",
           plot_height = int(1080*0.85),
           tools = TOOLS, plot_width = int(1920*0.85))
p.toolbar.active_scroll = p.select_one(WheelZoomTool)
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Mtb'

for ID in long_true_unique_IDs:
    ID_df = subset_df[subset_df['Unique_ID']==ID]
    cell_ID = ID_df['Cell_ID'].iloc[0]
    color = ID_df["color_codes"].iloc[0]
    source = ColumnDataSource(data=ID_df)
    p.line(source=source,
           x='Time_hours', 
           y='Intracellular_thresholded_Mtb_content_smooth',
           color=color, 
           line_width = 3.5, 
           alpha=0.8,
           muted_color=color, 
           muted_alpha=0.2, legend_label=f'Cell ID:{cell_ID}')

p.select_one(HoverTool).tooltips = [
    ('Time (hours)', '@x'),
    ('Mtb', '@y'),
    ('Cell ID', '@Cell_ID')
]

# p.legend.location = "top_left"
# p.legend.click_policy="mute"

output_file("mtb_time_series.html", title="Line Chart")
show(p)

# Adding images

Also the above needs fixing in some slight ways...

In [141]:
from macrohet import tile, dataio

In [142]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


In [320]:
image_dir = os.path.join(base_dir, 'macrohet_images/Images')#_8bit
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             set_plane='sum_proj',
#                              set_channel=1,
#                              set_time = 1,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()

### and masks

In [145]:
row, column

(3, 5)

In [146]:
### load tracks
with btrack.io.HDF5FileHandler(os.path.join(base_dir, 
                                            f'labels/macrohet_seg_model/{row,column}.h5'), 
                                   'r', 
                                   obj_type='obj_type_1'
                                   ) as reader:
    segmentation = reader.segmentation

[INFO][2023/06/15 01:57:37 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 5).h5...
[INFO][2023/06/15 01:57:47 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/06/15 01:57:47 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 5).h5


# Save out each frame instead of sc glimpses

So that I can crop on the fly... but this will mean that many sc seg outlines will be in one auto created glimpse

In [148]:
segmentation.shape

(75, 6048, 6048)

In [152]:
images

,Array,Chunk
Bytes,5.11 GiB,7.75 MiB
Shape,"(75, 2, 6048, 6048)","(1, 2, 2016, 2016)"
Count,17550 Tasks,675 Chunks
Type,uint8,numpy.ndarray


In [307]:
mask = segmentation[0]
gfp = images[0,0,...].compute().compute()
rfp = images[0,1,...].compute().compute()

In [308]:
v.add_image(gfp)
v.add_image(rfp)

<Image layer 'rfp [2]' at 0x7f83d2488970>

In [319]:
v.add_image(scaled_image)

<Image layer 'scaled_image [2]' at 0x7f83df679a30>

In [318]:
scaled_image = np.uint8(rgb >> 8)

In [313]:
[(rgb[...,i]/2**16)*2**8 for i in rgb[...,:]]

MemoryError: Unable to allocate 1.21 TiB for an array with shape (6048, 3, 6048, 6048) and data type uint16

In [312]:
rgb_8bit = [(rgb[...,i]/2**16)*2**8 for i in rgb[...,:]]

(6048, 6048, 3)

### 8bit

In [ ]:
109

In [323]:
for frame in tqdm(range(0, len(segmentation)), total = len(segmentation), desc = 'Iterating over frames'):
    # select relevant frames
    mask = segmentation[frame]
    gfp = images[frame,0,...].compute().compute()
    rfp = images[frame,1,...].compute().compute()
    # clip the images so that the contrast is more apparent
    contrast_lim_gfp = np.clip(gfp, 358, 5886)
    contrast_lim_rfp = np.clip(rfp, 480, 1300)
    # normalise them
    norm_gfp = cv2.normalize(contrast_lim_gfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
    norm_rfp = cv2.normalize(contrast_lim_rfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
    # create empty rgb
    rgb = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint16)
    # Set the green channel (GFP)
    rgb[:, :, 1] = norm_gfp
    # Set the magenta channel (RFP) and threshold to remove background
    rgb[:, :, 2] = norm_rfp
    rgb[:, :, 0] = norm_rfp
    # scale down to 8bit
    rgb = np.uint8(rgb >> 8)
    for instance in tqdm(range(1, np.max(mask)), total = np.max(mask), leave = False, desc = 'Isolating individual mask overlays'):
        instance_mask = (mask == instance).astype(np.uint8)
        contours, _ = cv2.findContours(instance_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(rgb, contours, -1, (0, 2**8, 2**8), thickness=2) # make 8bit
    #downsize image to reduce stoarge demands 
    rgb = cv2.resize(rgb, (rgb.shape[1] // 2, rgb.shape[0] // 2))
    #save out rgb mask overlay image
    output_fn = f'/home/dayn/temp_macrohet_image_mask_overlay/r0{row}c0{column}f0-p0--ch-sk{frame+1}fk1fl1.tiff'
    io.imsave(output_fn, rgb)

Iterating over frames:   0%|          | 0/75 [00:00<?, ?it/s]

Isolating individual mask overlays:   0%|          | 0/627 [00:00<?, ?it/s]

Isolating individual mask overlays:   0%|          | 0/638 [00:00<?, ?it/s]

Isolating individual mask overlays:   0%|          | 0/630 [00:00<?, ?it/s]

Isolating individual mask overlays:   0%|          | 0/625 [00:00<?, ?it/s]

...

Isolating individual mask overlays:   0%|          | 0/493 [00:00<?, ?it/s]

Isolating individual mask overlays:   0%|          | 0/490 [00:00<?, ?it/s]

Isolating individual mask overlays:   0%|          | 0/485 [00:00<?, ?it/s]

Isolating individual mask overlays:   0%|          | 0/481 [00:00<?, ?it/s]

Isolating individual mask overlays:   0%|          | 0/475 [00:00<?, ?it/s]

In [302]:
for frame in tqdm(range(0, len(segmentation)), total = len(segmentation)):
    # select relevant frames
    mask = segmentation[frame]
    gfp = images[frame,0,...].compute().compute()
    rfp = images[frame,1,...].compute().compute()
    # clip the images so that the contrast is more apparent
    contrast_lim_gfp = np.clip(gfp, 358, 5886)
    contrast_lim_rfp = np.clip(rfp, 480, 1300)
    # normalise them
    norm_gfp = cv2.normalize(contrast_lim_gfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
    norm_rfp = cv2.normalize(contrast_lim_rfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
    # create empty rgb
    rgb = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint16)
    # Set the green channel (GFP)
    rgb[:, :, 1] = norm_gfp
    # Set the magenta channel (RFP) and threshold to remove background
    rgb[:, :, 2] = norm_rfp
    rgb[:, :, 0] = norm_rfp
    for instance in tqdm(range(1, np.max(mask)), total = np.max(mask)):
        instance_mask = (mask == instance).astype(np.uint8)
        contours, _ = cv2.findContours(instance_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(rgb, contours, -1, (0, 2**16, 2**16), thickness=2)
    #save out rgb mask overlay image
    output_fn = f'/mnt/DATA/macrohet/macrohet_images/mask_overlay_images/r0{row}c0{column}f0-p0--ch-sk{frame+1}fk1fl1.tiff'
    io.imsave(output_fn, rgb)

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/627 [00:00<?, ?it/s]

  0%|          | 0/638 [00:00<?, ?it/s]

  0%|          | 0/630 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Add URL of images to subset_df

In [157]:
# Create a new column 'glimpse_url' with default values of None
subset_df['glimpse_url'] = None

# Iterate over the rows of the subset_df DataFrame
for index, row_ in subset_df.iterrows():
    cell_ID = row_['Cell_ID']
    online_fn = f'https://macrohet.s3.eu-west-2.amazonaws.com/{row},{column}/glimpse_{cell_ID}.{row}.{column}_t0.png'
#     output_fn = f'/mnt/DATA/macrohet/results/glimpses/interactive_plots/{row},{column}/glimpse_{cell_ID}.{row}.{column}_t0.png'
    subset_df.at[index, 'glimpse_url'] = online_fn


In [155]:
online_fn

'https://github.com/nthndy/macrohet/blob/main/downstream_analysis/plotting/interactive_plot_glimpses/3%2C5/glimpse_210.3.5_t0.png'

In [225]:
1.4949402023919043E-7*6048*1E6

904.1398344066238

# WIP plots

In [138]:
"""
This plot can be edited to show the different conditions and hide them upon clicking
"""

TOOLS = 'crosshair,save,pan,box_zoom,reset,wheel_zoom,hover'
p = figure(title="title",
           plot_height = int(1080*0.75),
           tools = TOOLS, plot_width = int(1920*0.75))
p.toolbar.active_scroll = p.select_one(WheelZoomTool)
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Mtb'

for ID in long_true_unique_IDs:
    ID_df = subset_df[subset_df['Unique_ID']==ID]
    cell_ID = ID_df['Cell_ID'].iloc[0]
    color = ID_df["color_codes"].iloc[0]
    source = ColumnDataSource(data=ID_df)
    p.line(source=source,
           x='Time_hours', 
           y='Intracellular_thresholded_Mtb_content_smooth',
           color=color, 
           line_width = 3.5, 
           alpha=0.8,
           muted_color=color, 
           muted_alpha=0.2, legend_label=f'Cell ID:{cell_ID}')

p.select_one(HoverTool).tooltips = [
    ('Time (hours)', '@x'),
    ('Mtb', '@y'),
    ('Cell ID', '@Cell_ID')
]

p.legend.location = "top_left"
p.legend.click_policy="mute"

output_file("mtb_time_series.html", title="Line Chart")
show(p)